In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import scipy.stats
from sampled import sampled
import seaborn as sns
import theano.tensor as tt
import pandas as pd

# Concentration of measure
We locate the typical set in higher dimensions. As dimension increases, the typical set of a gaussian becomes concentrated on a "narrower" region.

In [ ]:
for N in (2, 4, 8, 16, 32):
    x = scipy.stats.multivariate_normal(cov=np.eye(N) / N).rvs(size=10000)
    _ = sns.distplot(np.linalg.norm(x, axis=1))

plt.xlim(0, 2.5)
plt.ylim(0, 3.5)

plt.savefig('pics/concentration_of_measure.png', bbox_inches='tight')

In [ ]:
def jointplot(ary):
    """Helper to plot everything consistently"""
    sns.jointplot(*ary.T, alpha=0.1, stat_func=None, xlim=(-1.2, 1.2), ylim=(-1.2, 1.2))

In [ ]:
def pair_contour_plot(data):
    g = sns.PairGrid(df)
    g.map_diag(sns.kdeplot)
    g.map_offdiag(sns.kdeplot, cmap="Blues_d", n_levels=6);

# Unit Shell Distribution

https://chi-feng.github.io/mcmc-demo/app.html

In [ ]:
def tt_unit_shell_pdf(scale):
    def logp(x):
         return -tt.square((1 - x.norm(2)) / scale)
    return logp

@sampled
def unit_shell(dim=2, scale=0.001, **observed):
    testval = np.ones(dim) / np.linalg.norm(np.ones(dim))
    pm.DensityDist('unit_shell', logp=tt_unit_shell_pdf(scale), shape=dim, testval=testval)

In [ ]:
dim = 2
draws = 10000

with unit_shell(dim=dim, scale=0.01):
    step = pm.Metropolis()
    metropolis_sample = pm.sample(draws=draws, step=step, tune=500)

In [ ]:
df = pd.DataFrame(metropolis_sample.get_values('unit_shell'))
save = 0

sns.pairplot(df)
if save:
    plt.savefig('pics/shell_mrw_tuning_dim{}_scatter.png'.format(dim), 
                bbox_inches='tight')

pair_contour_plot(df)
if save:
    plt.savefig('pics/shell_mrw_tuning_dim{}_contour.png'.format(dim),
            bbox_inches='tight')

pm.traceplot(metropolis_sample, ['unit_shell'], figsize=(12,6))
if save:
    plt.savefig('pics/shell_mrw_tuning_dim{}_trace.png'.format(dim),
            bbox_inches='tight')

# Hamiltonian Methods

Let's try on something really simple, sampling a normal distribution in 1D

$$H(q, p) = -\log \pi(p | q) - \log \pi(q)$$
$$\frac{dq}{dt} = \frac{\partial H}{\partial p}$$
$$\frac{dp}{dt} = -\frac{\partial H}{\partial q}$$

Example:

Let $\pi(q) = \mathcal{N}(0, 1)$, and $\pi(p | q) = \mathcal{N}(0, 1),$ so 
$$ H(q, p) = C + p^2 + q^2.$$

We can manually compute the gradients for these distributions in order to plot the HMC paths in 2 dimensions: 1 for position, 1 for momentum.

In [ ]:
def gen_data(position, momentum, n=10000):
    """Generate a background density plot for the position and momentum distributions.  Not used for sampling."""
    q = position.rvs(n)
    p = momentum(q).rvs()
    return np.column_stack([q, p])

In [ ]:
def leapfrog(q, p, dHdq, dHdp, step_size, n_steps):
    """Perform the leapfrog integration.  
    
    Similar to the implementations in PyMC3, but 
    returns an array of all the points in the path
    for plotting.  It is a pretty general 
    implementation, in that it doesnt hardcode
    the potential or kinetic energies.
    
    Args:
        q: current position
        p: current momentum
        dHdq: see Hamilton's equations above
        dHdp: see Hamilton's equations above
        step_size: How big of a step to take
        n_steps: How many steps to take
    
    Returns:
        (`n_steps` x 2) numpy array showing the 
        path through (position, momentum) space 
        the Hamiltonian path took.
    """
    data = [[q, p]]
    p += 0.5 * step_size * -dHdq(q, p)
    q += step_size * dHdp(q, p)
    data.append([q, p])
    for _ in range(n_steps - 1):
        p += step_size * -dHdq(q, p)
        q += step_size * dHdp(q, p)
        data.append([q, p])
    p += 0.5 * step_size * -dHdq(q, p)
    return np.array(data)    

In [ ]:
def leapfrog_paths(position, momentum, dHdq, dHdp, n=10):
    """Get a number `n` of paths from the HMC sampler.

    This is not quite as general -- I hardcode a step 
    size of 0.01 and 100 steps, since a path length of 1
    looked pretty good (and *not* because it is the best
    choice).  Returns an iterator of plot data.
    """
    q = position.rvs()
    p = momentum(q).rvs()
    for _ in range(n):
        path = leapfrog(q, p, dHdq, dHdp, 0.01, 100)
        yield path
        q, _ = path[-1]
        p = momentum(q).rvs()

In [ ]:
position = scipy.stats.norm(0, 1)
momentum = lambda q: scipy.stats.norm(0, np.ones(q.shape))
dHdp = lambda q, p: 2 * p
dHdq = lambda q, p: 2 * q

In [ ]:
#  First plot a KDE of the joint pdf
X = gen_data(position, momentum)
g = sns.jointplot(*X.T, kind='kde', stat_func=None, xlim=(-3, 3), ylim=(-3, 3), alpha=0.5)

#  Now plot the Leapfrog paths on top of that
for path in leapfrog_paths(position, momentum, dHdq, dHdp, 15):
    g.ax_joint.plot(*path.T, linewidth=3)

# Unit Shell with Hamiltonian Monte Carlo

$$H(q, p) = -\log \pi(p | q) - \log \pi(q)$$
$$\frac{dq}{dt} = \frac{\partial H}{\partial p}$$
$$\frac{dp}{dt} = -\frac{\partial H}{\partial q}$$

Example:

Let $\log \pi(q) = -\left(\frac{1-|q|}{scale}\right)^2$, and $\pi(p | q) = \mathcal{N}(0, 1),$ so 
$$ H(q, p) = C + p^2 -\left(\frac{1-|q|}{scale}\right)^2$$

In [ ]:
dim = 5
draws = 1000

with unit_shell(dim=dim, scale=0.01):
    step = pm.HamiltonianMC(scaling=np.ones(dim) * 0.5)
    metropolis_sample = pm.sample(draws=draws, step=step)

In [ ]:
df = pd.DataFrame(metropolis_sample.get_values('unit_shell'))
save = 0

sns.pairplot(df)
if save:
    plt.savefig('pics/shell_mrw_tuning_dim{}_scatter.png'.format(dim), 
                bbox_inches='tight')

# pair_contour_plot(df)
if save:
    plt.savefig('pics/shell_mrw_tuning_dim{}_contour.png'.format(dim),
            bbox_inches='tight')

pm.traceplot(metropolis_sample, ['unit_shell'], figsize=(12,6))
if save:
    plt.savefig('pics/shell_mrw_tuning_dim{}_trace.png'.format(dim),
                bbox_inches='tight')

# Multivariate Gaussian

Finding a multivariate gaussian without adapatation and a "bad" initial guess sample

In [ ]:
def create_covariance(dim, off_diag=0.5, verbose=False):
    out = np.ones((dim, dim)) * off_diag
    np.fill_diagonal(out, 1) / dim
    if verbose:
        print(out)
    return out

In [ ]:
def tt_gaussian_pdf(mean, cov):
    mean = np.asarray(mean)
    cov = np.asarray(cov)
    dim = mean.shape[0]
    
    constant = -np.log((2*np.pi)**dim * np.linalg.det(cov))/2
    covinv = np.linalg.inv(cov)
    def logp(x):
        return constant - tt.dot(tt.dot((x - mean).T, covinv), (x - mean))/2
    return logp

@sampled
def gauss(mean=[0,0], cov=[[1,0],[0,1]], **observed):
    mean = np.asarray(mean)
    cov = np.asarray(cov)
    dim = mean.shape[0]
    testval = np.zeros(dim)
    pm.DensityDist('gauss', logp=tt_gaussian_pdf(mean, cov), shape=dim, testval=testval)

In [ ]:
dim = 5
mean = np.zeros(dim)
cov = np.eye(dim) / dim

starting_point = np.ones(dim) * 5

with gauss(mean=mean, cov=cov):
    step = pm.HamiltonianMC(scaling=cov, is_cov=True)
    metropolis_sample = pm.sample(draws=1000, step=step, start={'gauss': starting_point}, 
                                  tune=500, discard_tuned_samples=False)

In [ ]:
df = pd.DataFrame(metropolis_sample.get_values('gauss'))
sns.pairplot(df)
pm.traceplot(metropolis_sample, ['gauss'], figsize=(12,6))

# Banana

In [ ]:
def tt_banana_pdf(mean, cov, warp):
    mean = np.asarray(mean)
    cov = np.asarray(cov)
    dim = mean.shape[0]
    
    constant = -np.log((2*np.pi)**dim * np.linalg.det(cov))/2
    covinv = np.linalg.inv(cov)
    
    def logp(x):
        distortion = np.ones(dim) * warp * x[0]**2
        tt.set_subtensor(distortion[0], 0)
        return constant - tt.dot(tt.dot((x + distortion - mean).T, covinv), (x + distortion - mean))/2
    return logp

@sampled
def banana(mean=[0,0], cov=[[1,0],[0,1]], warp=0.9, **observed):
    mean = np.asarray(mean)
    cov = np.asarray(cov)
    dim = mean.shape[0]
    testval = np.zeros(dim)
    pm.DensityDist('banana', logp=tt_banana_pdf(mean, cov, warp), shape=dim, testval=testval)

In [ ]:
dim = 2
mean = np.zeros(dim)
cov = np.eye(dim)/dim
warp = 0.5

starting_point = np.zeros(dim)

with banana(mean=mean, cov=cov, warp=warp):
    step = pm.Metropolis()
    step = pm.NUTS()
    metropolis_sample = pm.sample(draws=1000, step=step, 
                                  start={'banana': starting_point}, 
                                  tune=500, discard_tuned_samples=True)

In [ ]:
df = pd.DataFrame(metropolis_sample.get_values('banana'))
sns.pairplot(df)

# pair_contour_plot(df)

pm.traceplot(metropolis_sample)